In [1]:
from keras import models
from keras import layers


Using TensorFlow backend.


In [ ]:
model  = models.Sequential()
#卷积层，参数意义分别为：
#经过这一层之后，特征图的个数，一个卷积核，产生一个特征图，第一层：32，说明有32个卷积核；第二层64，说明在第一层的特征图基础上，每张特征图有两个卷积核进行特征采集
#卷积核大小
#激活函数
#输入大小（只在开始的第一层有，后面不需要）
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(400,100,3)))
model.add(layers.MaxPool2D(2,2))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(3, activation='sigmoid'))


In [ ]:
#配置模型的损失函数、优化器、指标名称
from keras import optimizers

model.compile(loss='binary_crossentropy',          #损失函数
optimizer=optimizers.RMSprop(lr=1e-4),             #优化器
metrics=['acc'])                                   #指标名称


In [ ]:
#图片的训练路径和验证路径
train_dir = r'G:\useful_L\NEW_NET_3.0\train'
validation_dir = r'G:\useful_L\NEW_NET_3.0\val'


In [ ]:
#生成训练需要的图片和标签
from keras.preprocessing.image import ImageDataGenerator
#将图片大小调整到1以内，原先图片每个像素的格式为uint8，所以要除以255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

#根据目录的名称，生成对应的标签
#train_dir有Ⅱ型和Ⅲ型的图片
#每次生成batch_size数量的图片，图片大小为target_size
train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(400, 100),              #生成图片的大小
batch_size=30,                       #一次生成图片的数量
class_mode='categorical')                 #图片标签的类型

validation_generator = validation_datagen.flow_from_directory(
validation_dir,
target_size=(400, 100),              #生成图片的大小
batch_size=10,                       #一次生成图片的数量
class_mode='categorical')                 #图片标签的类型


In [ ]:
#开始训练
history = model.fit_generator(
train_generator,                           #通过生成器传入图片和标签
steps_per_epoch=100,                       #分100次传入，每次20张
epochs=50,                                 #总共训练40轮
validation_data=validation_generator,      #通过生成器传入图片和标签进行验证
validation_steps=90)                       #分90次传入，每次10张

In [ ]:
#绘制训练精度、验证精度
#绘制训练损失、验证损失
#python画图库，类似matlab的plot
import matplotlib.pyplot as plt
acc = history.history['acc']             #得到训练的指标数据
val_acc = history.history['val_acc']     #得到验证的指标数据
loss = history.history['loss']           #得到训练损失
val_loss = history.history['val_loss']   #得到验证损失
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.savefig('多分类1.0_accuracy_50.png')
plt.legend()                              #画图例
plt.figure()                              #另一张图
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.savefig('多分类1.0_loss_50.png')
plt.legend()
plt.show()                                #画图，最后加上

In [ ]:
# 保存每轮的精度和损失

file = open('多分类1.0_acc_loss_50.txt','a')
file.write('训练精度：')
for i in acc :
    file.write(str(i))
    file.write(" ")
file.write("\n")
file.write('验证精度：')
for i in val_acc :
    file.write(str(i))
    file.write(" ")


file.write("\n")
file.write('训练损失：')
for i in loss :
    file.write(str(i))
    file.write(" ")

file.write("\n")
file.write('验证损失：')
for i in val_loss :
    file.write(str(i))
    file.write(" ")

file.close()

In [ ]:
import os
import cv2 as cv
import numpy as np

III_dir = r'G:\useful_L\NEW_NET_2.1\val\III'
O_dir = r'G:\useful_L\NEW_NET_2.1\val\other'
n_dir = r'G:\useful_L\NEW_NET_2.1\val\noburst'

def my_image(path):
    out = []
    filenames = os.listdir(path)
    for filename in filenames:
        image = cv.imread(os.path.join(path, filename))

        image = cv.resize(image, (100, 400))
        image = image/255.0
        out.append(image)
    return np.array(out)

imgs_III = my_image(III_dir)
imgs_O = my_image(O_dir)
imgs_n = my_image(n_dir)
ret_III = model.predict_classes(imgs_III)
ret_O = model.predict_classes(imgs_O)
ret_n = model.predict_classes(imgs_n)

ret_III = ret_III.tolist()
ret_O = ret_O.tolist()
true = ret_III.count([0])
false = ret_O.count([0])
TPR = true/len(ret_III)
FPR = false/len(ret_O)
print("TPR is :{:f} ".format(TPR))
print("FPR is :{:f} ".format(FPR))



In [ ]:
model.save('多分类1.0.h5')


查看多分类模型输出的结果的输出样式
现在完成了III、没有爆发、其它类型  的分类网络，效果还好，96%-97%的准确率
是不是数据包丢失了


接下来，还要制作新的数据集
二型爆、三型爆、四型爆、一型爆的多分类模型
这里需要用到代价敏感损失函数
这里完成之后，再进行Inception-Net和SPP-Net的训练工作

总之 今天搞定代价敏感损失函数，制作出新的数据集，并训练出结果

把各种TPR\FPR做一遍
明天Inception-Net搭建并训练，看情况用不用SPP-NET，用了SPP之后，在实际应用中，
不用再进行图片的拉伸了


good job



In [6]:
import cv2
import numpy as np
from TestModel import TestModel
model_path = r'F:\SolarRadioBurst\classification\实验1\typeIIIMultiClassification\多分类1.3.h5'
path_test = r"G:\useful_L\NEW_NET_3.0\train\other\LM200104222255_cont+III+V_35_1801.png"

image = cv2.imread(path_test)
image = cv2.resize(image, (100, 400))
image = image/255.0
out = []
out.append(image)
out = np.array(out)
result = TestModel(model_path).predict_classes(out)
print(result)

# 关于predict_classes的说明，预测的结果是数字，表示对应的第几个文件夹


ERROR! Session/line number was not unique in database. History logging moved to new session 1177
[2]
